In [ ]:
import numpy as np
import scipy.stats as sc
import random as rd
import pandas as pd
import math

In [ ]:
# Overtopping
crest_level = 8.1
lowquality = 0
yf = 0.7 # Grass
alpha = 3 # 1 op x

# Other deterministic
g = 9.81
fetch = [2392,3128,3480,2992,1622,738,240,120,120,176,750,2002,2848,3084,2574,1762]
angle = [10.5,33,55.5,78,100.5,123,145.5,168,-169.5,-147,-124.5,-102,-79.5,-57,-34.5,-12]
winddir = ["NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "N"]
revetment = [[225,250],[100,120],[70,80],[40,50]]
p_occ_wind = [6.4,10.4,14.5,10.3,1.9,2.3,2.2,2.8,4.5,4.9,4.1,2.2,13.9,10.5,6.0,3.1]
alpha = math.degrees(np.arctan(alpha ** -1))

# Load Water level
data = np.loadtxt('Cumulative_density_function_water_level.txt', delimiter=' ', unpack=True)

# Load Wind
df = pd.read_excel (r'exc_wind.xlsx')
exc_vel = df["Wind velocity [m/s] "].tolist()
exc_wind = []
for i in range(16):
    exc_wind.append(1 - np.array(df[winddir[i]].tolist()))

    
    
# Monte Carlo
n = 1000000
l = 150
Pf = []

for i in range(l):
    fail = 0
    for n in range(n):
        # Generate boundary conditions
        j = rd.choices([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],p_occ_wind)[0]
        w = rd.random()
        u = np.interp(w, exc_wind[j], exc_vel)
        h = np.interp(w, data[1], data[0])
        hp = sc.norm.rvs(3.50, 0.1)
        b = sc.norm.rvs(2.35, 0.3)
        Rc = crest_level - np.array([h])
        d = h - b

        # If there is no water depth, there is no failure
        if(d < 0):
            continue

        # Calculate wave height and period
        F = (fetch[j] * g) / u**2
        D = (d * g) / u**2
        Hdak = 0.283 * np.tanh(0.53 * D**0.75) * np.tanh((0.0125 * F**0.42)/(np.tanh(0.53 * D**0.75)))
        Tdak = 2.4 * np.pi * np.tanh(0.833 * D**0.375) * np.tanh((0.077 * F**0.25)/(np.tanh(0.833 * D**0.375)))
        H = (Hdak * u**2) / g
        T = (Tdak * u) / g
        L = T * (g * d) ** 0.5
        em10 = np.tan(math.radians(alpha)) / ((H/L)**0.5)

        # Oblique waves, y_beta and height reduction
        y_beta = 1 - 0.0033 * np.absolute(angle[j])
        if(np.absolute(angle[j]) > 80):
            y_beta = 1 - 0.0033 * 80
        if(np.absolute(angle[j]) > 110):
            H = 0
        elif(np.absolute(angle[j]) > 80):
            H = H * (110 - np.absolute(angle[j])) / 30
            if(H < 0):
                H = 0
                if(Rc < 0):
                    continue;
        
        # Grass cover quality
        if(H > 2):
            wavecat = 2
        elif(H > 1):
            wavecat = 1
        else:
            wavecat = 0
        index = lowquality+wavecat
        sigma = (np.log(1 + (revetment[index][1]/revetment[index][0])**2))**0.5
        mu = np.log(revetment[index][0]) - 0.5 * sigma**2
        q = np.random.lognormal(mu, sigma);
        
        # Berm, vertical walls
        yb = 1 # No Berm
        yv = 1
        
        # Z-functions
        Z = 0
        if(Rc < 0):
            # Overflow
            Z = q - 0.54 * (g * np.absolute(Rc ** 3)) ** 0.5
        elif(H > 0):
            # Overtopping
            Z = q - (0.023 / (np.tan(math.radians(alpha)))) * yb * em10 * np.exp(-(((2.7 * Rc) / (yb * yf * y_beta * yb * em10 * H))**1.3)) * ((g * H**3) ** 0.5) * 10**3
            
        if(Z < 0):
            fail += 1
    if(fail > 0):
        Pf.append(fail/n)
    else:
        Pf.append(0)
    
    uitkomst = "FAIL"
    if(Pf[i] < 2.4e-5):
        uitkomst = "PASS"
    
    print(Pf[i], uitkomst)
print(np.sum(Pf) / l)